In [1]:
import os
import numpy as np
import torch as T
import torch.nn as nn
import torch.nn.functional as F #activation functions
import torch.optim as optim
from make_env import make_env

In [2]:
class MultiAgentReplayBuffer:
    def __init__(self,max_size,critic_dims,actor_dims,n_actions,n_agents,batch_size):
        self.mem_size=max_size
        self.mem_cntr=0
        self.n_agents=n_agents
        self.batch_size=batch_size
        self.n_actions=n_actions
        self.actor_dims=actor_dims
        
        self.state_memory=np.zeros((self.mem_size,critic_dims))
        self.new_state_memory=np.zeros((self.mem_size,critic_dims))
        self.reward_memory=np.zeros((self.mem_size,n_agents))
        self.terminal_memory=np.zeros((self.mem_size, n_agents), dtype=bool)
        
        self.init_actor_memory()
    
    def init_actor_memory(self):
        self.actor_state_memory=[]
        self.actor_new_state_memory=[]
        self.actor_action_memory=[]
        
        for i in range(n_agents):
            self.actor_state_memory.append(np.zeros((self.mem_size,self.actor_dims[i])))
            self.actor_new_state_memory.append(np.zeros((self.mem_size,self.actor_dims[i])))
            self.actor_action_memory.append(np.zeros((self.mem_size,self.n_actions)))
        
        
    def store_transition(self,raw_obs,state,action,reward,raw_obs_,state_,done):
        if self.mem_cntr % self.mem_size ==0 and self.mem_cntr > 0:
            self.init_actor_memory()
        index = self.mem_cntr % self.mem_size

        for agent_idx in range(self.n_agents):
            self.actor_state_memory[agent_idx][index]=raw_obs[agent_idx]
            self.actor_new_state_memory[agent_idx][index]=raw_obs[agent_idx]
            self.actor_action_memory[agent_idx][index]=action[agent_idx]

        self.state_memory[index] =state
        self.new_state_memory[index]=state_
        self.reward_memory[index]=reward
        self.terminal_memory[index]=done

        self.mem_cntr +=1

    def sample_buffer(self):
        max_mem=min(self.mem_cntr,self.mem_size)
        batch=np.random.choice(max_mem,self.batch_size,replace=False)

        states=self.state_memory[batch]
        rewards=self.reward_memory[batch]
        state_=self.new_state_memory[batch]
        terminal=self.terminal_memory[batch]

        actor_states=[]
        actor_new_states=[]
        actions=[]

        for agent_idx in range(self.n_agents):
            actor_states.append(self.actor_state_memory[agent_idx][batch])
            actor_new_states.append(self.actor_new_state_memory[agent_idx][batch])
            actions.append(self.actor_action_memory[agent_idx][batch])

        return actor_states,states,actions,rewards,actor_new_states,state_,terminal

    def ready(self):
        if self.mem_cntr >= self.batch_size:
            return True
        return False


In [3]:
class CriticNetwork(nn.Module):
    def __init__(self,beta,input_dims,fc1_dims,fc2_dims,n_agents,n_actions,name,chkpt_dir):
        super(CriticNetwork,self).__init__()
        
        self.chkpt_file=os.path.join(chkpt_dir,name)
        self.fc1=nn.Linear(input_dims+n_agents*n_actions,fc1_dims)
        self.fc2=nn.Linear(fc1_dims,fc2_dims)
        self.q=nn.Linear(fc2_dims,1)
        self.optimizer=optim.Adam(self.parameters(),lr=beta)
        self.device=T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
    
    def forward(self,state,action):
        x=F.relu(self.fc1(T.cat([state,action],dim=1)))
        x=F.relu(self.fc2(x))
        q=self.q(x)
        return q
    
    def save_checkpoint(self):
        T.save(self.state_dict(),self.chkpt_file)
    
    def load_chkpoint(self):
        self.load_state_dict(T.load(self.chkpt_file))
        
    
    


In [4]:
class ActorNetwork(nn.Module):
    def __init__(self,alpha,input_dims,fc1_dims,fc2_dims,n_actions,name,chkpt_dir):
        super(ActorNetwork,self).__init__()
        
        self.chkpt_file=os.path.join(chkpt_dir,name)
        self.fc1=nn.Linear(input_dims,fc1_dims)
        self.fc2=nn.Linear(fc1_dims,fc2_dims)
        self.pi=nn.Linear(fc2_dims,n_actions)
        
        self.optimizer=optim.Adam(self.parameters(),lr=alpha)
        self.device=T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)
    
    def forward(self,state):
        x=F.relu(self.fc1(state))
        x=F.relu(self.fc2(x))
        pi=T.softmax(self.pi(x),dim=1)
        
        return pi
    
    def save_checkpoint(self):
        T.save(self.state_dict(),self.chkpt_file)
    
    def load_chkpoint(self):
        self.load_state_dict(T.load(self.chkpt_file))
        
    
    

In [5]:
class Agent:
    def __init__(self,actor_dims,critic_dims,n_actions,agent_idx,chkpt_dir,alpha=0.01,beta=0.01,fc1=64,fc2=64,gamma=0.95,tau=0.01):
        self.gamma=gamma
        self.tau=tau
        self.n_actions=n_actions
        self.agent_name='agent_%s' % agent_idx
        
        self.actor=ActorNetwork(alpha,actor_dims,fc1,fc2,n_actions,chkpt_dir,self.agent_name+'_actor')
        self.critic=CriticNetwork(beta,critic_dims,fc1,fc2,n_agents,n_actions,chkpt_dir,self.agent_name+'_critic')
        self.target_actor=ActorNetwork(alpha,actor_dims,fc1,fc2,n_actions,chkpt_dir,self.agent_name+'_target_actor')
        self.target_critic=CriticNetwork(beta,critic_dims,fc1,fc2,n_agents,n_actions,chkpt_dir,self.agent_name+'_target_critic')
        
        self.update_network_parameters(tau=1)
    
    def update_network_parameters(self,tau=None):
        if tau is None:
             tau=self.tau
                           
        target_actor_params=self.target_actor.named_parameters()
        actor_params=self.actor.named_parameters()
        target_actor_state_dict=dict(target_actor_params)
        actor_state_dict=dict(actor_params)
        
        for name in actor_state_dict:
            actor_state_dict[name]=tau*actor_state_dict[name].clone() + (1-tau)*target_actor_state_dict[name].clone()
                        
        self.target_actor.load_state_dict(actor_state_dict)
        target_critic_params=self.target_critic.named_parameters()
        critic_params=self.critic.named_parameters()
        
        target_critic_state_dict=dict(target_critic_params)
        critic_state_dict=dict(critic_params)
        
        for name in critic_state_dict:
               critic_state_dict[name]=tau*critic_state_dict[name].clone() + (1-tau)*target_critic_state_dict[name].clone()
        self.target_critic.load_state_dict(critic_state_dict)
                           
    def choose_action(self,observation):
        state=T.tensor([observation],dtype=T.float).to(self.actor.device)
        actions=self.actor.forward(state)
        noise=T.rand(self.n_actions).to(self.actor.device)
        action=actions+noise

        return action.detach().cpu().numpy()[0]

    def save_models(self):
        self.actor.save_checkpoint()
        self.target.save_checkpoint()
        self.critic.save_checkpoint()
        self.target_critic.save_checkpoint()

    def load_models(self):
        self.actor.load_checkpoint()
        self.critic.load_checkpoint()
        self.target_actor.load_checkpoint()
        self.target_critic.load_checkpoint()






In [6]:
class MADDPG:
    def __init__(self,actor_dims,critic_dims,n_agents,n_actions,scenario='simple',alpha=0.01,beta=0.01,fc1=64,fc2=64,gamma=0.99,tau=0.01,chkpt_dir='tmp/maddpg/'):
        self.agents=[]
        self.n_agents=n_agents
        self.n_actions=n_actions
        chkpt_dir +=scenario
        
        for agent_idx in range(self.n_agents):
            self.agents.append(Agent(actor_dims[agent_idx],critic_dims,n_actions,agent_idx,alpha=alpha,beta=beta,chkpt_dir=chkpt_dir))
            
    def save_checkpoint(self):
        print('..saving checkpoint..')
        for agent in self.agents:
            agent.save_models()
            
    def load_checkpoint(self):
        print('..loading checkpoint..')
        for agent in self.agents:
            agent.load_models()
            
    def choose_action(self,raw_obs):
        actions=[]
        for agent_idx,agent in enumerate(self.agents):
            action=agent.choose_action(raw_obs[agent_idx])
            actions.append(action)
        return actions
    
    def learn(self,memory):
        if not memory.ready():
            return
    
        actor_states,states,actions,rewards,actor_new_states,states_,dones=memory.sample_buffer()

        device=self.agents[0].actor.device
        states=T.tensor(states,dtype=T.float).to(device)
        actions=T.tensor(actions,dtype=T.float).to(device)
        rewards=T.tensor(rewards,dtype=T.float).to(device)
        states_=T.tensor(states_,dtype=T.float).to(device)
        dones=T.tensor(dones).to(device)
        
        all_agents_new_actions=[]
        all_agents_new_mu_actions=[]
        old_agents_actions=[]
        
        for agent_idx,agent in enumerate(self.agents):
            new_states=T.tensor(actor_new_states[agent_idx],dtype=T.float).to(device)
            new_pi=agent.target_actor.forward(new_states)
            all_agents_new_actions.append(new_pi)
            mu_states=T.tensor(actor_states[agent_idx],dtype=T.float).to(device)
            pi=agent.actor.forward(mu_states)
            all_agents_new_mu_actions.append(pi)
            old_agents_actions.append(actions[agent_idx])
        
        new_actions = T.cat([acts for acts in all_agents_new_actions],dim=1)
        mu=T.cat([acts for acts in all_agents_new_mu_actions],dim=1)
        old_actions=T.cat([acts for acts in old_agents_actions],dim=1)
        
        for agent_idx,agent in enumerate(self.agents):
            critic_value_=agent.target_critic.forward(states_,new_actions).flatten()
            critic_value_[dones[:,0]]=0.0
            critic_value=agent.critic.forward(states,old_actions).flatten()
            target=rewards[:,agent_idx]+agent.gamma*critic_value_
            critic_loss=F.mse_loss(target,critic_value)
            agent.critic.optimizer.zero_grad()
            critic_loss.backward(retain_graph=True)
            agent.critic.optimizer.step()
            
            actor_loss=agent.critic.forward(states,mu).flatten()
            actor_loss=-T.mean(actor_loss)
            agent.actor.optimizer.zero_grad()
            actor_loss.backward(retain_graph=True)
            agent.actor.optimizer.step()
            
            agent.update_network_parameters()
 

In [7]:
           
def obs_list_to_state_vector(observation):
    state=np.array([])
    for obs in observation:
        state=np.concatenate([state,obs])
    return state
    
if __name__ == '__main__':
    scenario='simple'
    env=make_env(scenario)
    n_agents=env.n
    actor_dims=[]
    for i in range(n_agents):
        actor_dims.append(env.observation_space[i].shape[0])
    critic_dims=sum(actor_dims)
    n_actions=env.action_space[0].n
    maddpg_agents=MADDPG(actor_dims,critic_dims,n_agents,n_actions,fc1=64,fc2=64,alpha=0.01,beta=0.01,scenario=scenario,chkpt_dir='tmp/maddpg/')
    memory=MultiAgentReplayBuffer(1000000,critic_dims,actor_dims,n_actions,n_agents,batch_size=1024)

    PRINT_INTERVAL=500
    N_GAMES=30000
    MAX_STEPS=25
    total_steps=0
    score_history=[]
    evaluate=False
    best_score=0

    if evaluate:
        maddpg_agents.load_checkpoint()

    for i in range(N_GAMES):
        obs=env.reset()
        score=0
        done=[False]*n_agents
        episode_step=0
        while not any(done):
            if evaluate:
                env.render()
            actions=maddpg_agents.choose_action(obs)
            obs_,reward,done,info=env.step(actions)
            state=obs_list_to_state_vector(obs)
            state_=obs_list_to_state_vector(obs_)

            if episode_step>MAX_STEPS:
                done=[True]*n_agents
            memory.store_transition(obs,state,actions,reward,obs_,state_,done)

            if total_steps % 100 == 0 and not evaluate:
                maddpg_agents.learn(memory)

            obs=obs_

            score+=sum(reward)
            total_steps+=1
            episode_step+=1

        score_history.append(score)
        avg_score=np.mean(score_history[-100:])
        if not evaluate:
            if avg_score > best_score:
                maddpg_agents.save_checkpoint()
                best_score=avg_score

            if i % PRINT_INTERVAL ==0 and i>0:
                print('episode',i,'average_score{:.1f}'.format(avg_score))

/tmp/ipykernel_7751/3135724898.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  state=T.tensor([observation],dtype=T.float).to(self.actor.device)


episode 500 average_score-68.7
episode 1000 average_score-10.4
episode 1500 average_score-7.1
episode 2000 average_score-7.6
episode 2500 average_score-7.4
episode 3000 average_score-5.8
episode 3500 average_score-7.3
episode 4000 average_score-7.5
episode 4500 average_score-7.5
episode 5000 average_score-7.0
episode 5500 average_score-9.0
episode 6000 average_score-7.7
episode 6500 average_score-9.1
episode 7000 average_score-8.8
episode 7500 average_score-10.1
episode 8000 average_score-8.8
episode 8500 average_score-9.8
episode 9000 average_score-9.2
episode 9500 average_score-8.5
episode 10000 average_score-9.7
episode 10500 average_score-10.6
episode 11000 average_score-7.7
episode 11500 average_score-10.7
episode 12000 average_score-8.6
episode 12500 average_score-8.9
episode 13000 average_score-9.3
episode 13500 average_score-8.7
episode 14000 average_score-7.0
episode 14500 average_score-8.9
episode 15000 average_score-8.5
episode 15500 average_score-6.8
episode 16000 average_s